# Notebook 06b: Point Addition — The Geometry

**Module 06 -- Elliptic Curves**

---

**Motivating Question.** We know that an elliptic curve is a set of points satisfying $y^2 = x^3 + ax + b$. But a set of points is not a *group* until we define an operation. How do we "add" two points on a curve? The answer is a beautiful geometric construction: draw a line, find the third intersection, reflect. This **chord-and-tangent rule** turns the curve into an abelian group — the engine behind elliptic curve cryptography.

---

**Prerequisites.** You should be comfortable with:
- Elliptic curves over the reals and the Weierstrass equation (Notebook 06a)
- The point at infinity $\mathcal{O}$ and point negation (Notebook 06a)
- Group axioms: closure, associativity, identity, inverses (Module 01)

**Learning objectives.** By the end of this notebook you will be able to:
1. Describe the chord-and-tangent construction for adding two points.
2. Handle all special cases: distinct points, doubling, vertical lines, identity.
3. Derive the algebraic formulas for point addition and doubling.
4. Verify the group axioms computationally.
5. Visualise the addition operation step by step.

## 1. The Big Idea: Line, Intersect, Reflect

In Notebook 06a we saw that a line generically meets an elliptic curve in **three** points (by Bézout's theorem). This gives us a recipe for addition:

**To compute $P + Q$:**
1. Draw the line $\ell$ through $P$ and $Q$.
2. Find the third intersection point $R$ of $\ell$ with the curve.
3. Reflect $R$ across the $x$-axis: $P + Q = -R = (x_R, -y_R)$.

That's it! The reflection step is what makes the operation a group law (it ensures $\mathcal{O}$ is the identity).

| Step | Geometric action | Algebraic result |
|------|-----------------|------------------|
| 1 | Draw line through $P, Q$ | Compute slope $\lambda$ |
| 2 | Find third intersection $R$ | Solve cubic for $x_R$ |
| 3 | Reflect $R$ over $x$-axis | Negate $y_R$ |

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_curve(ax, a_val, b_val, xmin=-3, xmax=3):
    """Plot the elliptic curve y^2 = x^3 + ax + b."""
    x = np.linspace(xmin, xmax, 2000)
    rhs = x**3 + a_val * x + b_val
    mask = rhs >= 0
    y = np.sqrt(rhs[mask])
    ax.plot(x[mask], y, 'b-', linewidth=2)
    ax.plot(x[mask], -y, 'b-', linewidth=2)
    ax.axhline(y=0, color='gray', linewidth=0.3)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

# Visualise the full addition P + Q on y^2 = x^3 - 2x + 1
E = EllipticCurve(QQ, [-2, 1])
P = E(-1, 2)
Q = E(0, 1)
S = P + Q  # the result

# The third intersection R (before reflection) has same x as S, opposite y
R_x, R_y = S[0], -S[1]

fig, ax = plt.subplots(1, 1, figsize=(9, 7))
plot_curve(ax, -2, 1, xmin=-2.5, xmax=3.5)

# Draw line through P and Q, extended
px, py = float(P[0]), float(P[1])
qx, qy = float(Q[0]), float(Q[1])
slope = (qy - py) / (qx - px)
x_line = np.linspace(-2.5, 3.5, 100)
y_line = py + slope * (x_line - px)
ax.plot(x_line, y_line, 'r--', linewidth=1, alpha=0.7, label='Line through $P, Q$')

# Mark points
ax.plot(px, py, 'ro', markersize=10, zorder=5)
ax.annotate('$P$', (px, py), textcoords='offset points', xytext=(-15, 10), fontsize=14, fontweight='bold')
ax.plot(qx, qy, 'go', markersize=10, zorder=5)
ax.annotate('$Q$', (qx, qy), textcoords='offset points', xytext=(-15, 10), fontsize=14, fontweight='bold')

# Mark R (third intersection, before reflection)
rx, ry = float(R_x), float(R_y)
ax.plot(rx, ry, 'ms', markersize=10, zorder=5)
ax.annotate('$R$ (3rd intersection)', (rx, ry), textcoords='offset points', xytext=(10, 10), fontsize=11)

# Mark P + Q = -R (reflected)
sx, sy = float(S[0]), float(S[1])
ax.plot(sx, sy, 'k*', markersize=15, zorder=5)
ax.annotate('$P + Q = -R$', (sx, sy), textcoords='offset points', xytext=(10, -15), fontsize=14, fontweight='bold')

# Dashed vertical line showing reflection
ax.plot([rx, rx], [ry, sy], 'k:', linewidth=1.5, alpha=0.5)
ax.annotate('reflect', (rx, (ry + sy)/2), textcoords='offset points', xytext=(8, 0), fontsize=10, style='italic')

ax.set_title(r'Point Addition on $y^2 = x^3 - 2x + 1$: $P + Q$', fontsize=14)
ax.set_xlim(-2.5, 3.5)
ax.set_ylim(-4, 4)
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"P = {P}")
print(f"Q = {Q}")
print(f"R (3rd intersection) = ({R_x}, {R_y})")
print(f"P + Q = -R = {S}")

> **Checkpoint 1.** In the plot above, trace the three steps: (1) the red dashed line passes through $P$ and $Q$, (2) it hits the curve a third time at $R$, and (3) reflecting $R$ across the $x$-axis gives $P + Q$. Why is the reflection step necessary? (Hint: without it, the operation would not have $\mathcal{O}$ as identity.)

## 2. The Algebraic Formulas

Let $E: y^2 = x^3 + ax + b$ and let $P = (x_1, y_1)$, $Q = (x_2, y_2)$ be points on $E$ with $P \neq -Q$.

### Case 1: $P \neq Q$ (chord)

The slope of the line through $P$ and $Q$ is:
$$\lambda = \frac{y_2 - y_1}{x_2 - x_1}$$

### Case 2: $P = Q$ (tangent / point doubling)

We use implicit differentiation on $y^2 = x^3 + ax + b$ to get the tangent slope:
$$\lambda = \frac{3x_1^2 + a}{2y_1}$$

(This requires $y_1 \neq 0$; if $y_1 = 0$ then the tangent is vertical and $2P = \mathcal{O}$.)

### In both cases:

$$x_3 = \lambda^2 - x_1 - x_2$$
$$y_3 = \lambda(x_1 - x_3) - y_1$$

The result is $P + Q = (x_3, y_3)$.

| | Slope $\lambda$ | Then |
|---|---|---|
| $P \neq Q$ | $\frac{y_2 - y_1}{x_2 - x_1}$ | $x_3 = \lambda^2 - x_1 - x_2$, $y_3 = \lambda(x_1 - x_3) - y_1$ |
| $P = Q$ | $\frac{3x_1^2 + a}{2y_1}$ | Same formulas for $x_3, y_3$ (with $x_2 = x_1$) |

In [ ]:
# Implement point addition by hand and compare with SageMath
def ec_add_manual(E, P, Q):
    """Add P + Q on elliptic curve E using the explicit formulas."""
    O = E(0)  # point at infinity
    
    # Identity cases
    if P == O:
        return Q
    if Q == O:
        return P
    
    x1, y1 = P[0], P[1]
    x2, y2 = Q[0], Q[1]
    a = E.a4()  # coefficient of x in short Weierstrass form
    
    # Inverse case: P + (-P) = O
    if x1 == x2 and y1 == -y2:
        return O
    
    # Compute slope
    if P == Q:
        # Tangent (doubling)
        lam = (3 * x1^2 + a) / (2 * y1)
    else:
        # Secant (addition)
        lam = (y2 - y1) / (x2 - x1)
    
    # Compute result
    x3 = lam^2 - x1 - x2
    y3 = lam * (x1 - x3) - y1
    
    return E(x3, y3)

# Test
E = EllipticCurve(QQ, [-2, 1])
P = E(-1, 2)
Q = E(0, 1)

result_manual = ec_add_manual(E, P, Q)
result_sage = P + Q

print(f"P = {P}")
print(f"Q = {Q}")
print(f"P + Q (manual)  = {result_manual}")
print(f"P + Q (SageMath) = {result_sage}")
print(f"Match? {result_manual == result_sage}")

> **Misconception alert.** "Point addition on elliptic curves is the same as adding coordinates." Absolutely not! $(x_1, y_1) + (x_2, y_2) \neq (x_1 + x_2, y_1 + y_2)$. The elliptic curve group law is a *geometric* operation defined by line intersections, which happens to produce algebraic formulas involving the curve equation.

## 3. Point Doubling ($P + P = 2P$)

When $P = Q$, there is no "line through two distinct points" — instead, we take the **tangent line** to the curve at $P$. This tangent generically meets the curve in one more point (with multiplicity), which we reflect to get $2P$.

In [ ]:
# Visualise point doubling
E = EllipticCurve(QQ, [-2, 1])
P = E(0, 1)
two_P = 2 * P

# Tangent slope at P
x1, y1 = float(P[0]), float(P[1])
a_val = -2
lam = (3 * x1**2 + a_val) / (2 * y1)

# R is the third intersection (before reflection)
R_x, R_y = float(two_P[0]), float(-two_P[1])

fig, ax = plt.subplots(1, 1, figsize=(9, 7))
plot_curve(ax, -2, 1, xmin=-2, xmax=4)

# Tangent line
x_line = np.linspace(-2, 4, 100)
y_line = y1 + lam * (x_line - x1)
ax.plot(x_line, y_line, 'r--', linewidth=1, alpha=0.7, label='Tangent at $P$')

# Mark P
ax.plot(x1, y1, 'ro', markersize=10, zorder=5)
ax.annotate('$P$', (x1, y1), textcoords='offset points', xytext=(-15, 10), fontsize=14, fontweight='bold')

# Mark R
ax.plot(R_x, R_y, 'ms', markersize=10, zorder=5)
ax.annotate('$R$', (R_x, R_y), textcoords='offset points', xytext=(10, 10), fontsize=12)

# Mark 2P
sx, sy = float(two_P[0]), float(two_P[1])
ax.plot(sx, sy, 'k*', markersize=15, zorder=5)
ax.annotate('$2P = -R$', (sx, sy), textcoords='offset points', xytext=(10, -15), fontsize=14, fontweight='bold')

# Reflection line
ax.plot([R_x, R_x], [R_y, sy], 'k:', linewidth=1.5, alpha=0.5)

ax.set_title(r'Point Doubling: $2P$ via tangent line', fontsize=14)
ax.set_xlim(-2, 4)
ax.set_ylim(-5, 5)
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"P = {P}")
print(f"Tangent slope λ = {lam}")
print(f"2P = {two_P}")

In [ ]:
# Verify the doubling formula step by step
E = EllipticCurve(QQ, [-2, 1])
P = E(0, 1)
x1, y1 = P[0], P[1]
a = E.a4()

print("Step-by-step doubling of P = (0, 1) on y^2 = x^3 - 2x + 1:")
print(f"  a = {a}")

lam = (3*x1^2 + a) / (2*y1)
print(f"  λ = (3·{x1}² + ({a})) / (2·{y1}) = {lam}")

x3 = lam^2 - x1 - x1
print(f"  x₃ = {lam}² - {x1} - {x1} = {x3}")

y3 = lam * (x1 - x3) - y1
print(f"  y₃ = {lam}·({x1} - {x3}) - {y1} = {y3}")

print(f"\n  2P = ({x3}, {y3})")
print(f"  SageMath: 2*P = {2*P}")
print(f"  Match? {E(x3, y3) == 2*P}")

> **Checkpoint 2.** In the doubling formula, the slope is $\lambda = \frac{3x_1^2 + a}{2y_1}$. What happens when $y_1 = 0$? Geometrically, the tangent at a point with $y = 0$ is vertical — the line goes to $\mathcal{O}$, so $2P = \mathcal{O}$. Such points have **order 2** in the group.

## 4. All the Special Cases

The chord-and-tangent rule has several special cases that we must handle:

| Case | Geometric situation | Result |
|------|-------------------|--------|
| $P + \mathcal{O}$ | $P$ plus identity | $P$ |
| $P + (-P)$ | Vertical line through $P$ and $-P$ | $\mathcal{O}$ |
| $P + Q$ ($P \neq \pm Q$) | Secant line through $P, Q$ | Use chord formula |
| $P + P$ ($y_1 \neq 0$) | Tangent line at $P$ | Use doubling formula |
| $P + P$ ($y_1 = 0$) | Vertical tangent | $\mathcal{O}$ |

In [ ]:
# Demonstrate all special cases
E = EllipticCurve(QQ, [-1, 0])  # y^2 = x^3 - x
O = E(0)
P = E(1, 0)   # a point with y = 0
Q = E(-1, 0)  # another point with y = 0
R = E(0, 0)   # yet another

print("Curve: y^2 = x^3 - x")
print(f"Points with y=0 (order 2): {P}, {Q}, {R}")
print()

# Case 1: P + O = P
print(f"Case 1 (identity): P + O = {P + O}  [should be P = {P}]")

# Case 2: P + (-P) = O
# For P = (1,0), -P = (1,-0) = (1,0) = P, so P + P = O
print(f"Case 2 (inverse):  P + (-P) = P + P = {P + P}  [should be O, since y=0]")

# Case 3: P + Q (distinct, P ≠ -Q)
print(f"Case 3 (chord):    P + Q = {P + Q}  [P=(1,0), Q=(-1,0)]")

# Case 4: Doubling a general point
S = E(-1, 0)
print(f"Case 5 (y=0 double): S + S = {S + S}  [S=(-1,0) has order 2]")

In [ ]:
# Visualise the vertical line case (P + (-P) = O)
E = EllipticCurve(QQ, [-2, 1])
P = E(-1, 2)
neg_P = -P

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: P + (-P) = O (vertical line)
plot_curve(ax1, -2, 1, xmin=-2.5, xmax=3)
px, py = float(P[0]), float(P[1])
ax1.plot(px, py, 'ro', markersize=10, zorder=5)
ax1.annotate('$P$', (px, py), textcoords='offset points', xytext=(10, 5), fontsize=14)
ax1.plot(px, -py, 'go', markersize=10, zorder=5)
ax1.annotate('$-P$', (px, -py), textcoords='offset points', xytext=(10, -15), fontsize=14)
ax1.plot([px, px], [-4, 4], 'r--', linewidth=1, alpha=0.5)
ax1.set_title('$P + (-P) = \\mathcal{O}$\n(vertical line → infinity)', fontsize=13)
ax1.set_xlim(-2.5, 3)
ax1.set_ylim(-4, 4)

# Right: P + Q (general case)
Q = E(0, 1)
S = P + Q
plot_curve(ax2, -2, 1, xmin=-2.5, xmax=4)
qx, qy = float(Q[0]), float(Q[1])
slope = (qy - py) / (qx - px)
x_line = np.linspace(-2.5, 4, 100)
y_line = py + slope * (x_line - px)
ax2.plot(x_line, y_line, 'r--', linewidth=1, alpha=0.5)
ax2.plot(px, py, 'ro', markersize=10, zorder=5)
ax2.annotate('$P$', (px, py), textcoords='offset points', xytext=(-15, 10), fontsize=14)
ax2.plot(qx, qy, 'go', markersize=10, zorder=5)
ax2.annotate('$Q$', (qx, qy), textcoords='offset points', xytext=(-15, 10), fontsize=14)
sx, sy = float(S[0]), float(S[1])
ax2.plot(sx, sy, 'k*', markersize=15, zorder=5)
ax2.annotate('$P+Q$', (sx, sy), textcoords='offset points', xytext=(10, -15), fontsize=14)
ax2.set_title('$P + Q$ (general chord case)', fontsize=13)
ax2.set_xlim(-2.5, 4)
ax2.set_ylim(-5, 5)

plt.tight_layout()
plt.show()

## 5. Verifying the Group Axioms

For the elliptic curve points to form a group under $+$, we need:

1. **Closure:** If $P, Q \in E$, then $P + Q \in E$.
2. **Identity:** $P + \mathcal{O} = P$ for all $P$.
3. **Inverses:** For each $P$, there exists $-P$ with $P + (-P) = \mathcal{O}$.
4. **Associativity:** $(P + Q) + R = P + (Q + R)$ for all $P, Q, R$.

Closure, identity, and inverses are clear from the construction. Associativity is the hard part — the algebraic proof is tedious, so let us verify it computationally.

In [ ]:
# Verify group axioms on a concrete curve
E = EllipticCurve(QQ, [-2, 1])
O = E(0)
P = E(-1, 2)
Q = E(0, 1)
R = E(1, 0)

print("=== Group Axiom Verification ===")
print(f"Curve: {E}")
print(f"P = {P}, Q = {Q}, R = {R}")
print()

# 1. Closure
S = P + Q
print(f"1. Closure: P + Q = {S} ∈ E? {S in E}")

# 2. Identity
print(f"2. Identity: P + O = {P + O} == P? {P + O == P}")
print(f"            O + P = {O + P} == P? {O + P == P}")

# 3. Inverses
neg_P = -P
print(f"3. Inverse:  -P = {neg_P}")
print(f"            P + (-P) = {P + neg_P} == O? {P + neg_P == O}")

# 4. Associativity
lhs = (P + Q) + R
rhs = P + (Q + R)
print(f"4. Associativity:")
print(f"   (P + Q) + R = {lhs}")
print(f"   P + (Q + R) = {rhs}")
print(f"   Equal? {lhs == rhs}")

# 5. Commutativity (bonus — elliptic curve groups are abelian)
print(f"5. Commutativity:")
print(f"   P + Q = {P + Q}")
print(f"   Q + P = {Q + P}")
print(f"   Equal? {P + Q == Q + P}")

In [ ]:
# Stress-test associativity with random points over a finite field
p = 10007
E_fp = EllipticCurve(GF(p), [-2, 1])

num_tests = 100
all_pass = True
for i in range(num_tests):
    P = E_fp.random_point()
    Q = E_fp.random_point()
    R = E_fp.random_point()
    if (P + Q) + R != P + (Q + R):
        print(f"FAIL at test {i}: P={P}, Q={Q}, R={R}")
        all_pass = False
        break

print(f"Associativity test: {num_tests} random triples over F_{p} — {'ALL PASSED' if all_pass else 'FAILED'}")

## 6. Building a Sequence: $P, 2P, 3P, \ldots$

Just as we computed $g, g^2, g^3, \ldots$ in $\mathbb{Z}/p\mathbb{Z}^*$ (Module 05), we can compute repeated additions $P, 2P, 3P, \ldots$ on an elliptic curve. If $P$ generates a cyclic subgroup of order $n$, then $nP = \mathcal{O}$ and the sequence cycles back.

---

> **Bridge from Module 01.** In Module 01, the "power table" of a generator $g$ in $(\mathbb{Z}/p\mathbb{Z}^*, \times)$ listed $g^1, g^2, \ldots, g^{p-1} = 1$. Here we build the analogous "multiple table" for a point $P$ in $(E, +)$: $1P, 2P, \ldots, nP = \mathcal{O}$. The notation is additive instead of multiplicative, but the structure is identical.

In [ ]:
# Compute multiples of a point
E = EllipticCurve(QQ, [-2, 1])
P = E(-1, 2)

print(f"Multiples of P = {P} on {E}:\n")
print(f"{'k':>4}  {'kP':>40}")
print("-" * 48)
for k in range(1, 15):
    kP = k * P
    coords = f"({kP[0]}, {kP[1]})" if kP != E(0) else "O (identity)"
    print(f"{k:>4}  {coords:>40}")

In [ ]:
# Visualise multiples of P on the curve
E = EllipticCurve(GF(97), [-2, 1])
P = E(0, 1)
n = P.order()

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Plot all multiples
xs, ys = [], []
for k in range(1, n):
    kP = k * P
    xs.append(int(kP[0]))
    ys.append(int(kP[1]))

ax.scatter(xs, ys, c=range(len(xs)), cmap='viridis', s=30, zorder=5)

# Highlight first few multiples
colors = ['red', 'green', 'blue', 'purple', 'orange']
for k in range(1, 6):
    kP = k * P
    ax.plot(int(kP[0]), int(kP[1]), 'o', color=colors[k-1], markersize=10, zorder=6)
    ax.annotate(f'{k}P', (int(kP[0]), int(kP[1])), textcoords='offset points',
                xytext=(5, 5), fontsize=9, color=colors[k-1])

ax.set_title(f'Multiples of $P = (0, 1)$ on $E(\\mathbb{{F}}_{{{97}}})$\nOrder of $P$: {n}', fontsize=13)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Order of P: {n}")
print(f"Verification: {n}·P = {n * P}")

Notice how the multiples of $P$ appear to "scatter" across the finite field — there is no visible pattern. This is the elliptic curve analogue of the "scramble" we saw with discrete exponentiation in Module 05. Given $kP$, recovering $k$ is the **Elliptic Curve Discrete Logarithm Problem (ECDLP)**.

> **Crypto foreshadowing.** The ECDLP — given $P$ and $Q = kP$, find $k$ — is believed to be even harder than the classical DLP in $\mathbb{Z}/p\mathbb{Z}^*$. The best known attack is Pollard's rho, which runs in $O(\sqrt{n})$ time with no sub-exponential shortcuts. This is why 256-bit EC keys provide 128-bit security.

## 7. Derivation of the Addition Formulas (Optional)

For those who want to see *why* the formulas work, here is the derivation.

**Setup:** Let $P = (x_1, y_1)$ and $Q = (x_2, y_2)$ with $x_1 \neq x_2$. The line through them is $y = \lambda x + \nu$ where $\lambda = \frac{y_2 - y_1}{x_2 - x_1}$ and $\nu = y_1 - \lambda x_1$.

**Substituting** into $y^2 = x^3 + ax + b$:

$(\lambda x + \nu)^2 = x^3 + ax + b$

$x^3 - \lambda^2 x^2 + (a - 2\lambda\nu)x + (b - \nu^2) = 0$

This cubic has three roots: $x_1, x_2, x_3$. By **Vieta's formulas**, the sum of roots equals the coefficient of $x^2$ (with sign):

$$x_1 + x_2 + x_3 = \lambda^2 \implies x_3 = \lambda^2 - x_1 - x_2$$

Then $y_3 = \lambda(x_1 - x_3) - y_1$ (evaluating the line at $x_3$ and negating for the reflection).

In [ ]:
# Verify Vieta's formulas
E = EllipticCurve(QQ, [-2, 1])
P = E(-1, 2)
Q = E(0, 1)
S = P + Q  # = (x3, y3) after reflection

x1, y1 = P[0], P[1]
x2, y2 = Q[0], Q[1]
lam = (y2 - y1) / (x2 - x1)

# x3 from Vieta: x1 + x2 + x3 = λ²
x3_vieta = lam^2 - x1 - x2
print(f"λ = {lam}")
print(f"Vieta: x₁ + x₂ + x₃ = λ² → x₃ = {x3_vieta}")
print(f"Actual x₃ = {S[0]}")
print(f"Match? {x3_vieta == S[0]}")
print(f"\nSum of roots: {x1} + {x2} + {x3_vieta} = {x1 + x2 + x3_vieta} = λ² = {lam^2}")

> **Checkpoint 3.** Why do we use Vieta's formulas instead of solving the cubic directly? Because Vieta's gives us $x_3$ using only addition and subtraction of the known roots $x_1, x_2$ — no need for the cubic formula or numerical methods. This is what makes the group law computationally efficient.

## 8. Exercises

### Exercise 1 (Worked): Manual Point Addition

**Problem.** On $E: y^2 = x^3 + 1$ over $\mathbb{Q}$, compute $P + Q$ where $P = (0, 1)$ and $Q = (2, 3)$.

**Solution.**

Step 1: Compute the slope.
$$\lambda = \frac{y_2 - y_1}{x_2 - x_1} = \frac{3 - 1}{2 - 0} = \frac{2}{2} = 1$$

Step 2: Compute $x_3$.
$$x_3 = \lambda^2 - x_1 - x_2 = 1^2 - 0 - 2 = -1$$

Step 3: Compute $y_3$.
$$y_3 = \lambda(x_1 - x_3) - y_1 = 1 \cdot (0 - (-1)) - 1 = 1 - 1 = 0$$

So $P + Q = (-1, 0)$.

In [ ]:
# Exercise 1 — verification
E = EllipticCurve(QQ, [0, 1])  # y^2 = x^3 + 1
P = E(0, 1)
Q = E(2, 3)
result = P + Q
print(f"P + Q = {result}")
print(f"Expected: (-1, 0)")
print(f"Match? {result == E(-1, 0)}")

# Interesting: (-1, 0) has order 2!
R = E(-1, 0)
print(f"\n2·(-1, 0) = {2*R}  (order 2 point!)")

### Exercise 2 (Guided): Point Doubling by Hand

**Problem.** On $E: y^2 = x^3 - x + 1$ over $\mathbb{Q}$, compute $2P$ where $P = (1, 1)$.

*Steps:*
1. Compute $\lambda = \frac{3x_1^2 + a}{2y_1}$ with $a = -1$.
2. Compute $x_3 = \lambda^2 - 2x_1$.
3. Compute $y_3 = \lambda(x_1 - x_3) - y_1$.
4. Verify with SageMath.

In [ ]:
# Exercise 2 — fill in the TODOs
E = EllipticCurve(QQ, [-1, 1])  # y^2 = x^3 - x + 1
P = E(1, 1)
x1, y1 = P[0], P[1]
a = E.a4()

# TODO 1: Compute the tangent slope
# lam = ???

# TODO 2: Compute x3
# x3 = ???

# TODO 3: Compute y3
# y3 = ???

# TODO 4: Verify
# print(f"Manual: 2P = ({x3}, {y3})")
# print(f"SageMath: 2*P = {2*P}")

### Exercise 3 (Independent): Exploring the Group Structure

**Problem.**
1. On $E: y^2 = x^3 + 1$ over $\mathbb{Q}$, start with $P = (2, 3)$ and compute $2P, 3P, 4P, 5P, 6P$. At what point does $kP = \mathcal{O}$? (This is the order of $P$.)
2. The point $(-1, 0)$ has order 2 (since $y = 0$). Find $P + (-1, 0)$ for $P = (0, 1)$ and $P = (2, 3)$. Is there a pattern?
3. Verify associativity for three specific points of your choice on $E: y^2 = x^3 - 2x + 1$.

In [ ]:
# Exercise 3 — write your solution here


## Summary

| Concept | Key Fact |
|---------|----------|
| **Addition rule** | Draw line through $P, Q$ → find 3rd intersection $R$ → reflect: $P + Q = -R$ |
| **Chord formula** | $\lambda = \frac{y_2 - y_1}{x_2 - x_1}$, then $x_3 = \lambda^2 - x_1 - x_2$ |
| **Doubling formula** | $\lambda = \frac{3x_1^2 + a}{2y_1}$, same $x_3, y_3$ formulas |
| **Identity** | $P + \mathcal{O} = P$ — the point at infinity is the identity |
| **Inverses** | $-P = (x, -y)$ and $P + (-P) = \mathcal{O}$ |
| **Vieta's shortcut** | $x_3 = \lambda^2 - x_1 - x_2$ comes from sum-of-roots, avoiding the cubic formula |

We now have a complete group law for elliptic curves over any field. In the next notebook, we move from the continuous real line to **finite fields** $\mathbb{F}_p$, where the curve becomes a discrete set of points — and where cryptography actually happens.

---

**Next:** [06c — Curves over Finite Fields](06c-curves-over-finite-fields.ipynb)